# Anomaly Detection - Polk County, Iowa

Anomaly or outlier detection is essentially finding patterns that do not conform to expected behavior. There are several approaches to anomaly detection that are based on either statistical properties, clustering, classification, Principal Component Analysis (PCA), or subsampling. In this notebook we will look at an **autoencoder network** for anomaly or outlier detection. An autoencoder is a neural-net based, unsupervised learning model that is used to learn low-dimensional features that captures some structure underlying the high-dimensional input data.

## Lab Scenario

Groundwater level is an important metric, especially for agriculture states such as Iowa. One of the metrics [U.S. Geological Survey (USGS)](https://www.usgs.gov/) monitors is **depth to water level in feet below the land**. In this lab we will use a synthetic dataset that models certain scenarios for Polk County, Iowa. The three key weather-related metrics we will be using are:

- water-level (depth to water level in feet below the land)
- temperature
- humidity

The data is generated daily using realistic monthly averages for Polk County, Iowa, for the years 2017 – 2019. The data is generated daily for each of the 92 different sensors/locations within Polk County, Iowa – 3 years x 365 days x 92 sensors = 100,740 total sets of data.

We are going to be using 2 copies of the dataset for years 2017 -2019: 

1. Normal conditions for the county.
2. A gradual build up dry conditions in one of the regions in Polk County, Iowa over the months of June and July 2019.

The goal of this notebook is to develop an approach to monitor a group of sensors based on their proximity to each other to predict regional anomalies in real-time. We will be grouping the sensors in 6 different location-based clusters as identified by the previous notebook. Thus, in for model training, we will use cluster_id, along with month, temperature, humidity, and water level as our features.

To train an autoencoder model that learns the structures in the input data in this more complex scenario will need significant compute resources. In this notebook we will build and train the autoencoder model on local compute and thus at first, we will work with a subset of the data. Once the anomaly detection model and approach are established, in the next notebook we will be leveraging the compute resources provided by Azure Machine Learning service to train the model on the entire dataset. The subset data used in this notebook will range from May 2019 – August 2019 that encompasses the anomalous period of June-July 2019.

## Outline

1. **Setup**: Import required libraries, load the datasets, and create data subsets.

2. **Define and Train the Autoencoder Network on local compute**: Use Keras to define and train the autoencoder model.

3. **Establish criteria for anomalies**: Define approaches and thresholds for detecting anomalies based on the trained autoencoder model.

4. **Predict anomalies**: Used in the trained autoencoder model, make predictions to identify anomalies.

5. **Principal Component Analysis**: Apply PCA on the encoded dataset and visualize the data representation at lower dimensions.

## Setup

### Import required libraries 

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import os
import math
import timeit
from IPython.display import display, HTML, Image, SVG
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
np.random.seed(437)
print("pandas version: {} numpy version: {}".format(pd.__version__, np.__version__))

import sklearn
from sklearn import preprocessing
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn_pandas import DataFrameMapper
from sklearn.cluster import KMeans

import keras
import tensorflow
from keras.layers import Input, Dropout
from keras.layers.core import Dense 
from keras.models import Model, Sequential, load_model
from keras import regularizers
from keras.models import model_from_json

from numpy.random import seed
from tensorflow import set_random_seed

print("keras version: {} tensorflow version: {} sklearn version: {}".format(keras.__version__, 
                                                                        tensorflow.__version__, sklearn.__version__))

%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

print('importing libraries done!')

**Helper method to display a pandas dataframe**

In [ ]:
def display_dataframe(df_in):
    s = df_in.style.set_properties(**{'text-align': 'left'})
    s.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
    display(HTML(s.render()))

### Load the Datasets

In [ ]:
normal_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
              'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/anomaly_detection/normal_multi.xlsx')

gradual_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
               'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/anomaly_detection/gradual_multi.xlsx')

normal_df = pd.read_excel(normal_url)
gradual_df = pd.read_excel(gradual_url)

print('Size of dataset: {} rows'.format(len(normal_df)))
print('Done loading datasets!')

### Create Dataset Subsets

In [ ]:
normal_df = normal_df.loc[lambda d: (d.date >= '2019-05-01') & (d.date <= '2019-08-31'), :]
gradual_df = gradual_df.loc[lambda d: (d.date >= '2019-05-01') & (d.date <= '2019-08-31'), :]
print('Size of subset dataset: {} rows'.format(len(normal_df)))
print('Done creating subset datasets!')

## Define and Train the Autoencoder Network on local compute

### Preprocess Input Data

Select **cluster_id**, **month**, **temperature**, **humidity**, and **water level** as our features for the network.

In [ ]:
feature_cols = ['cluster_id', 'month', 'temperature', 'humidity', 'water_level']
categorical = ['cluster_id', 'month']
numerical = ['temperature', 'humidity', 'water_level']

numeric_transformations = [([f], Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())])) for f in numerical]
    
categorical_transformations = [([f], OneHotEncoder(handle_unknown='ignore', sparse=False)) for f in categorical]

transformations = categorical_transformations + numeric_transformations

clf = Pipeline(steps=[('preprocessor', DataFrameMapper(transformations))])

X = clf.fit_transform(normal_df[feature_cols])
X_train = X
np.random.shuffle(X_train)

### Define the Autoencoder Network Architecture

In [ ]:
seed(10)
set_random_seed(50)
act_func = 'elu'

input_ = Input(shape=(X_train.shape[1],))
x = Dense(100, activation=act_func)(input_)
x = Dense(50, activation=act_func)(x)
x = Dense(25, activation=act_func)(x)
encoder = Dense(12, activation=act_func, name='feature_vector')(x)
x = Dense(25, activation=act_func)(encoder)
x = Dense(50, activation=act_func)(x)
x = Dense(100, activation=act_func)(x)
output_ = Dense(X_train.shape[1], activation=act_func)(x)

model = Model(input_, output_)
lr = 0.001
opt = keras.optimizers.Adam(lr=lr)
model.compile(loss='mse', optimizer=opt)

encoder_model = Model(inputs=model.input, outputs=model.get_layer('feature_vector').output)
encoder_model.compile(loss='mse', optimizer='adam')

print('')
print(model.summary())
print('')

### Train the Autoencoder Model

In [ ]:
epochs = 100
batch_size = 16

print('epochs: ', epochs)
print('batch_size: ', batch_size)
print('')

def schedule(epoch_number, current_lr):
    lr = current_lr
    if (epoch_number < 25):
        lr = 0.001
    if (epoch_number >= 25) & (epoch_number < 35):
        lr = 0.0005
    if (epoch_number >= 35) & (epoch_number < 50):
        lr = 0.0003
    if (epoch_number >= 50) & (epoch_number < 60):
        lr = 0.0001
    if (epoch_number >= 60) & (epoch_number < 70):
        lr = 0.00008
    if (epoch_number >= 70) & (epoch_number < 80):
        lr = 0.00006
    if (epoch_number >= 80) & (epoch_number < 90):
        lr = 0.00004
    if (epoch_number >= 90) & (epoch_number < 100):
        lr = 0.00002
    if (epoch_number >= 100) & (epoch_number < 125):
        lr = 0.000009
    if (epoch_number >= 125) & (epoch_number < 150):
        lr = 0.000007
    if (epoch_number >= 150) & (epoch_number < 175):
        lr = 0.000005
    if (epoch_number >= 175) & (epoch_number < 200):
        lr = 0.000001
    return lr

lr_sch = keras.callbacks.LearningRateScheduler(schedule, verbose=1)

print("Model training starting...")
start_time = timeit.default_timer()
history = model.fit(X_train, X_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    validation_split=0.2, 
                    callbacks=[lr_sch], 
                    verbose=1)
elapsed_time = timeit.default_timer() - start_time
print("Model training completed.")
print('Elapsed time (min): ', round(elapsed_time/60.0,0))

**Save the models**

In [ ]:
os.makedirs('./output/models', exist_ok=True)
model.save(os.path.join('./output/models', 'anomaly_detection_multi_full_model.h5'))
encoder_model.save(os.path.join('./output/models', 'anomaly_detection_multi_encoder_model.h5'))

with open(os.path.join('./output/models', 'history.txt'), 'w') as f:
    f.write(str(history.history))
    
print("Models saved in ./output/models folder")
print("Saving model files completed.")

**Load the models**

Run this cell if you want to load previously trained models

In [ ]:
#model = load_model('./output/models/anomaly_detection_multi_full_model.h5')
#encoder_model = load_model('./output/models/anomaly_detection_multi_encoder_model.h5')
#print('Models loaded!')

## Establish Criteria for Anomalies

The autoencoder network is trained using normal data where it first compresses the input data and then reconstructs the input data. During training the network learns the interactions between various input variables under normal conditions and learns to reconstruct the input variables back to their original values. The reconstruction error is the error is reproducing back the original input values. We will be using `Mean Absolute Error` as our measure for the reconstruction error. The basic idea behind anomaly detection is that the reconstruction error using the trained network for anomalous inputs will be higher than what is typically observed with normal data. 

Thus, one of the parameters we need to understand is the **threshold for the reconstruction error** that identifies anomalous input data.

### Make predictions and compute reconstruction errors for the normal dataset

Next, we will make predictions on the normal dataset, compute the reconstruction error for individual set of inputs, and look that the upper and lower bounds for the reconstruction errors.

In [ ]:
X_train = clf.transform(normal_df[feature_cols]) # Keep the order, X_train used for training was shuffled
X_pred = model.predict(X_train)
loss_mae = np.mean(np.abs(X_pred-X_train), axis = 1)
normal_df['loss_mae'] = loss_mae
stats = normal_df.loss_mae.describe()
print(('Max loss mae: {}').format(stats['max']))

### Visualize the reconstruction errors for the normal dataset

It appears that the threshold value of `0.003` is a reasonable cutoff to identify anomalous input data.

In [ ]:
# Setup upper_bound for anomalous reconstruction error
upper_bound = 0.003

In [ ]:
plt.figure(figsize=(7, 5))

upper_boundary = upper_bound * np.ones(len(normal_df.date.unique()))

plt.plot_date(normal_df.date, normal_df.loss_mae, markersize=0.5)
plt.plot(normal_df.date.unique(), upper_boundary, color='r')

plt.xticks(fontsize=10, rotation=45);

plt.show()

### Visualize the reconstruction errors for the gradual datasets

Make predictions on the gradual dataset, compute the reconstruction error for individual set of inputs. You will see a ramp up in the reconstruction error for some set of data points around June-August 2019.

In [ ]:
X_gradual = clf.transform(gradual_df[feature_cols])
X_gradual_pred = model.predict(X_gradual)
loss_mae_gradual = np.mean(np.abs(X_gradual-X_gradual_pred), axis = 1)

gradual_df['loss_mae'] = loss_mae_gradual

In [ ]:
plt.figure(figsize=(7, 5))

upper_boundary = upper_bound * np.ones(len(gradual_df.date.unique()))

plt.plot_date(gradual_df.date, gradual_df.loss_mae, markersize=0.5)
plt.plot(gradual_df.date.unique(), upper_boundary, color='r')

plt.xticks(fontsize=10, rotation=45);

plt.show()

### Visualize the reconstruction errors for the various region clusters in the gradual datasets

**Is there a lower error threshold we can monitor to detect the potential anomaly earlier in the time scale?**

In [ ]:
cluster_df = gradual_df.groupby(['date', 'cluster_id'])['water_level', 'loss_mae'].mean()
cluster_df.reset_index(drop=False, inplace=True)

In [ ]:
cluster_upper_bound = 0.0015

In [ ]:
plt.figure(figsize=(7, 5))

cluster_upper_boundary = cluster_upper_bound * np.ones(len(cluster_df.date.unique()))

plt.plot_date(cluster_df.date, cluster_df.loss_mae, markersize=0.5)
plt.plot(cluster_df.date.unique(), cluster_upper_boundary, color='r')

plt.xticks(fontsize=10, rotation=45);

plt.show()

## Predict Anomalies

With the two established thresholds: **0.004** for point anomalies and **0.0015** for cluster anomalies, we will add the two types of predictions to our data sets, standard point anomalies (`anomaly_std`), and anomalies based on cluster averages (`anomaly_cluster`).

In [ ]:
def isAnomaly(date, cluster_id):
    loss = cluster_df.loc[lambda x: (x.date == date) & (x.cluster_id == cluster_id)]['loss_mae'].values[0]
    return (True if loss >= cluster_upper_bound else False)

**This cell will take around 30 seconds to complete**

In [ ]:
start_time = timeit.default_timer()
gradual_df['anomaly_std'] = gradual_df.loss_mae.apply(lambda x: True if x > upper_bound else False)
gradual_df['anomaly_cluster'] = gradual_df.apply(lambda x: isAnomaly(x.date, x.cluster_id), axis = 1)
elapsed_time = timeit.default_timer() - start_time
print('Elapsed time (seconds): ', round(elapsed_time))

**Review Anomalies in the Gradual dataset**

The table shows that regional anomalies are predicted for **cluster 1 / North region**. Furthermore, the `anomaly_cluster` starts on June 21st 2019 almost 17 days before the reconstruction error (`loss_mae`) exceeds the normal threshold.

In [ ]:
display_dataframe(gradual_df[(gradual_df.anomaly_cluster == True)])

### Review the reconstruction error for the North region

In [ ]:
f, ax = plt.subplots(figsize=(8, 5))

gradual_test_df = cluster_df.loc[lambda d: (d.date >= '2019-05-01') & (d.date <= '2019-08-31') & 
                                 (d.cluster_id == 1), :]

upper_boundary = cluster_upper_bound * np.ones(len(gradual_test_df))

ax.plot(gradual_test_df.loss_mae.values)
ax.plot(upper_boundary, color='r')

ax.set_title('Gradual Dataset')
ax.set_ylabel('Mean Absolute Error');

### Visualize Anomalies in the Observed Water Levels in the North region

Next, we will visualize the anomalies in the measured water levels during the anomalous period (June-August 2019). As you can see the cluster anomalies are established well before the point anomalies show persistance.

In [ ]:
gradual_test_df = gradual_df.loc[lambda d: (d.date.dt.year == 2019) & (d.cluster_id == 1) & 
                                ((d.date.dt.month == 6) | (d.date.dt.month == 7) | (d.date.dt.month == 8)), :]

In [ ]:
f, ax = plt.subplots(2, 1, sharey=True, sharex=True, figsize=(7, 5))

colors_g_1 = ['red' if value == True else 'blue' for value in gradual_test_df.anomaly_std.values]
size_g_1 = [10 if value == True else 5 for value in gradual_test_df.anomaly_std.values]
colors_g_2 = ['red' if value == True else 'blue' for value in gradual_test_df.anomaly_cluster.values]
size_g_2 = [10 if value == True else 5 for value in gradual_test_df.anomaly_cluster.values]

ax[0].scatter(gradual_test_df.date, gradual_test_df.water_level, s = size_g_1, c = colors_g_1)
ax[1].scatter(gradual_test_df.date, gradual_test_df.water_level, s = size_g_2, c = colors_g_2)

ax[0].set_title('Gradual Dataset - anomaly_std')
ax[1].set_title('Gradual Dataset - anomaly_cluster')

ax[0].set_ylabel('Water Level')
ax[1].set_ylabel('Water Level')

from matplotlib.patches import Patch
from matplotlib.lines import Line2D

legend_elements = [Line2D([0], [0], marker='o', color='w', label='Normal', markerfacecolor='b', markersize=5), 
                  Line2D([0], [0], marker='o', color='w', label='Anomaly', markerfacecolor='r', markersize=5)]

ax[0].legend(handles=legend_elements, frameon=False)
#ax[1].legend(handles=legend_elements, frameon=False)

ax[1].xaxis.set_ticks(['2019-06-01', '2019-06-15', '2019-07-01', '2019-07-15', 
                       '2019-08-01', '2019-08-15', '2019-08-31'])

plt.xticks(fontsize=10, rotation=25);

f.tight_layout(rect=[0, 0.03, 1, 0.95])

## Principal Component Analysis

Generate the top N principal components of the encoded representation of the input data for the gradual dataset during the anomalous period.

In [ ]:
gradual_anomalies = gradual_df.loc[lambda d: (d.date.dt.year == 2019) & (d.cluster_id == 1) & 
                                   ((d.date.dt.month == 5) | (d.date.dt.month == 6) | 
                                    (d.date.dt.month == 7) | (d.date.dt.month == 8)), :]
gradual_anomalies_encoded = encoder_model.predict(clf.transform(gradual_anomalies[feature_cols]))

Generate principal components for **N = [2, 3, 4, 5]**

In [ ]:
pca_components = [2, 3, 4, 5]
gradual_anomalies_pca = []

def pca_analysis(input, results, anomaly_type):
    for comp in pca_components: 
        pca = PCA(n_components = comp)
        pca_result = pca.fit_transform(input)
        print('{} - Cumulative explained variation for {} principal components: {}'.format(
            anomaly_type, comp, np.sum(pca.explained_variance_ratio_)))
        results.append(pca_result)

pca_analysis(gradual_anomalies_encoded, gradual_anomalies_pca, 'Gradual anomalies')

### Visualize the Principal Components for N = 3

Visualize the top 3 principal components of the encoded representation of the input data.

Looking at the 3-D plots, it appears that the second principal component is largely sufficient to predict anomalies. The anomalous or near-anomalous data lies in the region **x_2 > 1**.

In [ ]:
X_embedded1 = pd.DataFrame(gradual_anomalies_pca[1], columns=['X','Y', 'Z'])
X_embedded1['State'] = np.where(gradual_anomalies.anomaly_std, 'Failure', 'Normal')

X_embedded2 = pd.DataFrame(gradual_anomalies_pca[1], columns=['X','Y', 'Z'])
X_embedded2['State'] = np.where(gradual_anomalies.anomaly_cluster, 'Failure', 'Normal')


from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(10, 5))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')

ax1.set_title('Anomoly Std', y=-0.12)
ax2.set_title('Anomoly Cluster', y=-0.12)

colors_1 = ['red' if value == 'Failure' else 'blue' for value in X_embedded1.State.values]
ax1.scatter(X_embedded1.X.values, X_embedded1.Y.values, X_embedded1.Z.values, c=colors_1)

colors_2 = ['red' if value == 'Failure' else 'blue' for value in X_embedded2.State.values]
ax2.scatter(X_embedded2.X.values, X_embedded2.Y.values, X_embedded2.Z.values, c=colors_2)

#start, end = ax2.get_xlim()
#print(start, end)

ax1.xaxis.set_ticks([-1, 0, 1, 2])
ax1.yaxis.set_ticks([-1, 0, 1, 2])
ax1.zaxis.set_ticks([-1, -0.5, 0, 0.5, 1])

ax2.xaxis.set_ticks([-1, 0, 1, 2])
ax2.yaxis.set_ticks([-1, 0, 1, 2])
ax2.zaxis.set_ticks([-1, -0.5, 0, 0.5, 1])

from matplotlib.patches import Patch
from matplotlib.lines import Line2D

legend_elements = [Line2D([0], [0], marker='o', color='w', label='Normal', markerfacecolor='b', markersize=5), 
                  Line2D([0], [0], marker='o', color='w', label='Anomaly', markerfacecolor='r', markersize=5)]

ax1.legend(handles=legend_elements, loc='upper left', frameon=False)
ax2.legend(handles=legend_elements, loc='upper left', frameon=False)

plt.show()